#Final Evaluation

# Importing Dependencies


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.7.0+cu126 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Found existing installation: torch 2.7.0+cu126
Uninstalling torch-2.7.0+cu126:
  Successfully uninstalled torch-2.7.0+cu126
Found existing installation: torchvision 0.22.0+cu126
Uninstalling torchvision-0.22.0+cu126:
  Successfully uninstalled torchvision-0.22.0+cu126
Found existing installation: torchaudio 2.7.0+cu126
Uninstalling torchaudio-2.7.0+cu126:
  Successfully uninstalled torchaudio-2.7.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.7.0%2Bcu126-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.22.0%2Bcu126-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchaudio-2.7.0%2Bcu126-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.6 kB)
Using cached https://download.pytorch.org/whl/cu126/torch-2.7.0%2Bcu126-cp311-cp311-manylinux_2_28_x86_64.whl (867.0 MB)
Using cache

In [ ]:
import torch
import numpy as np
import transformers
import bitsandbytes as bnb
import xformers
import accelerate
import peft
import datasets
import trl
import unsloth

print("Torch version:", torch.__version__)
print("NumPy version:", np.__version__)
print("Transformers version:", transformers.__version__)
print("BitsAndBytes version:", bnb.__version__ if hasattr(bnb, "__version__") else "Version not found")
print("Accelerate version:", accelerate.__version__)
print("Peft version:", peft.__version__)
print("Datasets version:", datasets.__version__)
print("TRL version:", trl.__version__)
print("xformers version:", xformers.__version__)

<ipython-input-4-238ee508f917>:10: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Torch version: 2.7.0+cu126
NumPy version: 2.0.2
Transformers version: 4.51.3
BitsAndBytes version: 0.45.5
Accelerate version: 1.6.0
Peft version: 0.15.2
Datasets version: 3.6.0
TRL version: 0.15.2
xformers version: 0.0.30


In [ ]:
!ls /content/drive/MyDrive/NLP-GIT/lora_model/

adapter_config.json	   special_tokens_map.json  tokenizer.model
adapter_model.safetensors  tokenizer_config.json
README.md		   tokenizer.json


#  Loading Fine-tuned Model

In [ ]:
from unsloth import FastLanguageModel
base_model_path = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_path,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)


==((====))==  Unsloth 2025.4.8: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from peft import PeftModel
adapter_path = "/content/drive/MyDrive/NLP-GIT/lora_model"
model = PeftModel.from_pretrained(model, adapter_path)


# Sample Response Generation

In [ ]:
prompt = """
### Input:
I'm going to vacation.

### Response:
"""

inputs = tokenizer(prompt.strip(), return_tensors="pt").to("cuda")

with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=100)

print(tokenizer.decode(output[0], skip_special_tokens=True))

### Input:
I'm going to vacation.

### Response:
That's great to hear! Where are you planning to go on vacation? I'd be happy to help you plan or make recommendations if you need any assistance. Let me know if you have any specific preferences or requirements, such as budget, duration, or type of vacation (beach, adventure, cultural, etc.). If you don't have any particular destination in mind, I can suggest some popular vacation spots or destinations based on current trends or seasonal recommendations. Let me


#Loading Test data

In [ ]:
from datasets import load_dataset, DatasetDict
dataset = load_dataset("talalmuzaffar/empathetic_dataset_with_correct_context")
split_dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)
val_test_split = split_dataset['test'].train_test_split(test_size=0.5, seed=42)
test_dataset = val_test_split['test']


README.md:   0%|          | 0.00/392 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/962k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
!pip install tqdm

#  Generating Predictions

In [ ]:
from tqdm import tqdm
results = []
for idx, example in enumerate(tqdm(test_dataset, desc="Processing test samples")):
    input_text = example["Input"]
    reference_output = example["Response"]

    prompt = f"""
    ### Input:
    {input_text}

    ### Response:
    """

    inputs = tokenizer(prompt.strip(), return_tensors="pt").to("cuda")
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=100)
    predicted_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    results.append({
        "input": input_text,
        "reference": reference_output,
        "predicted": predicted_text
    })

    # Optionally print every 10 samples (if you want log lines)
    if idx % 10 == 0:
        print(f"Processed {idx + 1}/{len(test_dataset)}")


Processing test samples:   0%|          | 1/800 [00:09<2:08:16,  9.63s/it]

Processed 1/800


Processing test samples:   1%|▏         | 11/800 [01:13<1:22:55,  6.31s/it]

Processed 11/800


Processing test samples:   3%|▎         | 21/800 [02:19<1:19:42,  6.14s/it]

...

Processing test samples:  98%|█████████▊| 781/800 [1:16:33<01:50,  5.83s/it]

Processed 781/800


Processing test samples:  99%|█████████▉| 791/800 [1:17:30<00:55,  6.20s/it]

Processed 791/800


Processing test samples: 100%|██████████| 800/800 [1:18:09<00:00,  5.86s/it]


In [ ]:
import pandas as pd


In [ ]:
df_results = pd.DataFrame(results)


In [ ]:
df_results.head(2)

,input,reference,predicted
0,I was once in a grocery store and the fire ala...,I think once was 4 and the other was 2. I help...,It sounds like you had a moment of connection ...
1,I found my old N64 and hooked the system up. I...,Yeah it does! I am so happy it does but one ga...,I'm glad to hear that you've rediscovered your...


# Evaluating

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b74f4d5cfdf223df68fa5a2884405f65f1a22c0992cb9a526555de28cb0c31fb
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # This is often needed too


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
import evaluate

# Init rouge
rouge = evaluate.load("rouge")

# Initialize metric lists
bleu_1 = []
bleu_2 = []
bleu_3 = []
bleu_4 = []
meteor_scores = []
rouge1 = []
rouge2 = []
rougeL = []

# Loop over each row
for i, row in df_results.iterrows():
    reference = row["reference"]
    predicted = row["predicted"]

    # BLEU
    smoothie = SmoothingFunction().method4
    bleu_1.append(
        sentence_bleu([reference.strip().split()], predicted.strip().split(), weights=(1, 0, 0, 0), smoothing_function=smoothie)
    )
    bleu_2.append(
        sentence_bleu([reference.strip().split()], predicted.strip().split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
    )
    bleu_3.append(
        sentence_bleu([reference.strip().split()], predicted.strip().split(), weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
    )
    bleu_4.append(
        sentence_bleu([reference.strip().split()], predicted.strip().split(), weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    )

    # METEOR (safe)
    try:
        m_score = meteor_score([reference.strip().split()], predicted.strip().split())
        meteor_scores.append(m_score)
    except Exception as e:
        print(f"METEOR error at row {i}: {e}")
        continue

    # ROUGE
    rouge_result = rouge.compute(predictions=[predicted], references=[reference])
    rouge1.append(rouge_result["rouge1"])
    rouge2.append(rouge_result["rouge2"])
    rougeL.append(rouge_result["rougeL"])


In [ ]:
print("\n=== Average BLEU Scores ===")
print(f"BLEU-1: {sum(bleu_1)/len(bleu_1):.4f}")
print(f"BLEU-2: {sum(bleu_2)/len(bleu_2):.4f}")
print(f"BLEU-3: {sum(bleu_3)/len(bleu_3):.4f}")
print(f"BLEU-4: {sum(bleu_4)/len(bleu_4):.4f}")

if meteor_scores:
    print("\n=== Average METEOR ===")
    print(f"METEOR: {sum(meteor_scores)/len(meteor_scores):.4f}")
else:
    print("\n=== Average METEOR ===")
    print("No METEOR scores computed.")

print("\n=== ROUGE Scores ===")
print(f"ROUGE-1: {sum(rouge1)/len(rouge1):.4f}")
print(f"ROUGE-2: {sum(rouge2)/len(rouge2):.4f}")
print(f"ROUGE-L: {sum(rougeL)/len(rougeL):.4f}")



=== Average BLEU Scores ===
BLEU-1: 0.0540
BLEU-2: 0.0241
BLEU-3: 0.0152
BLEU-4: 0.0097

=== Average METEOR ===
METEOR: 0.1178

=== ROUGE Scores ===
ROUGE-1: 0.1135
ROUGE-2: 0.0182
ROUGE-L: 0.0880


In [ ]:
df_results.to_csv("mistral_F_evaluation_results.csv", index=False)


In [ ]:
import bert_score

# Extract predicted and reference lists
predictions = df_results["predicted"].tolist()
references = df_results["reference"].tolist()

# Run BERTScore (default model: roberta-large)
P, R, F1 = bert_score.score(predictions, references, lang="en", verbose=True)

# Print average scores
print("\n=== BERTScore ===")
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1:        {F1.mean().item():.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/24 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/13 [00:00<?, ?it/s]

done in 12.98 seconds, 61.64 sentences/sec

=== BERTScore ===
Precision: 0.8262
Recall:    0.8520
F1:        0.8387


In [ ]:
!pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.4 MB/s eta 0:00:00


In [ ]:
import evaluate
chrf = evaluate.load("chrf")
chrf_result = chrf.compute(predictions=df_results["predicted"].tolist(),
                           references=df_results["reference"].tolist())

print(f"ChrF: {chrf_result['score']:.4f}")

ChrF: 18.0778


In [ ]:
def distinct_n(corpus, n):
    all_ngrams = set()
    total_ngrams = 0
    for sentence in corpus:
        tokens = sentence.strip().split()
        ngrams = list(zip(*[tokens[i:] for i in range(n)]))
        all_ngrams.update(ngrams)
        total_ngrams += len(ngrams)
    return len(all_ngrams) / (total_ngrams + 1e-8)

predictions = df_results["predicted"].tolist()

print(f"Distinct-1: {distinct_n(predictions, 1):.4f}")
print(f"Distinct-2: {distinct_n(predictions, 2):.4f}")


Distinct-1: 0.1231
Distinct-2: 0.4318


In [ ]:
input_lens = df_results["input"].apply(lambda x: len(x.strip().split()))
ref_lens = df_results["reference"].apply(lambda x: len(x.strip().split()))
pred_lens = df_results["predicted"].apply(lambda x: len(x.strip().split()))

print(f"Avg input length: {input_lens.mean():.2f}")
print(f"Avg reference length: {ref_lens.mean():.2f}")
print(f"Avg predicted length: {pred_lens.mean():.2f}")


Avg input length: 18.40
Avg reference length: 13.76
Avg predicted length: 63.91


# Toxicity Analysis using Unitary Toxic-BERT
We use `unitary/toxic-bert` to detect whether any of the model's responses are toxic (with a threshold of 0.5). This ensures safety and appropriateness of the generated outputs.

In [ ]:
from transformers import pipeline
toxicity_classifier = pipeline("text-classification", model="unitary/toxic-bert")

toxic_counts = 0
for pred in df_results["predicted"]:
    result = toxicity_classifier(pred)[0]
    if result['label'] == 'toxic' and result['score'] > 0.5:
        toxic_counts += 1

print(f"Toxic responses: {toxic_counts}/{len(df_results)}")


config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Toxic responses: 0/800


Result of evaluations


| Metric                    | Score        | Description                                        |
|---------------------------|--------------|----------------------------------------------------|
| **BLEU-1**                | 0.0540       | Unigram overlap                                    |
| **BLEU-2**                | 0.0241       | Bigram overlap                                     |
| **BLEU-3**                | 0.0152       | Trigram overlap                                    |
| **BLEU-4**                | 0.0097       | 4-gram overlap                                     |
| **METEOR**                | 0.1178       | Semantic + lexical match                           |
| **ROUGE-1**               | 0.1135       | Unigram overlap recall                             |
| **ROUGE-2**               | 0.0182       | Bigram overlap recall                              |
| **ROUGE-L**               | 0.0880       | Longest common subsequence                         |
| **BERTScore (Precision)** | 0.8262       | Token-level semantic match                         |
| **BERTScore (Recall)**    | 0.8520       | Recall of semantic similarity                      |
| **BERTScore (F1)**        | 0.8387       | F1 of semantic similarity                          |
| **Distinct-1**            | 0.1231       | Diversity – unique unigrams                        |
| **Distinct-2**            | 0.4318       | Diversity – unique bigrams                         |
| **Avg Input Length**      | 18.40 tokens | Average length of input prompts                    |
| **Avg Reference Length**  | 13.76 tokens | Average ground truth response length               |
| **Avg Predicted Length**  | 63.91 tokens | Average model output length                        |
| **Toxic Responses**       | 0 / 800      | Number of responses classified as toxic            |
